In [6]:
import numpy as np
import altair as alt
from meshcat.servers.zmqserver import start_zmq_server_as_subprocess
import pydot
from IPython.display import display, SVG
from ipywidgets import Text, Textarea, Layout
import plotly.express as px
from pydrake.all import (
    AddMultibodyPlantSceneGraph, AngleAxis, BasicVector, ConnectMeshcatVisualizer, 
    DiagramBuilder, FindResourceOrThrow, Integrator, JacobianWrtVariable, 
    LeafSystem, MultibodyPlant, MultibodyPositionToGeometryPose, Parser, 
    PiecewisePolynomial, PiecewiseQuaternionSlerp, Quaternion, RigidTransform, 
    RollPitchYaw, RotationMatrix, SceneGraph, Simulator, TrajectorySource
)
from pydrake.examples.manipulation_station import ManipulationStation
from pydrake.multibody.jupyter_widgets import MakeJointSlidersThatPublishOnCallback

In [7]:
proc, zmq_url, web_url = start_zmq_server_as_subprocess()

In [4]:
def gripper_forward_kinematics_example():
    builder = DiagramBuilder()

    # TODO: Replace this with a simple model directive of iiwa+wsg (no clutter bins)
    station = builder.AddSystem(ManipulationStation())
    station.SetupClutterClearingStation()
    station.Finalize()

    frames_to_draw = {"iiwa": {"iiwa_link_1", "iiwa_link_2", "iiwa_link_3", "iiwa_link_4", "iiwa_link_5", "iiwa_link_6", "iiwa_link_7"},
                      "gripper": {"body"}}
    meshcat = ConnectMeshcatVisualizer(builder,
        station.get_scene_graph(),
        output_port=station.GetOutputPort("pose_bundle"),
        zmq_url=zmq_url,
        frames_to_draw=frames_to_draw,
        axis_length=0.3,
        axis_radius=0.01)

    diagram = builder.Build()
    context = diagram.CreateDefaultContext()

    xyz = Text(value="", description="gripper position (m): ", layout=Layout(width='500px'), style={'description_width':'initial'})
    rpy = Text(value="", description="gripper roll-pitch-yaw (rad): ", layout=Layout(width='500px'), style={'description_width':'initial'})
    plant = station.get_multibody_plant()

    gripper = plant.GetBodyByName("body")
    def pose_callback(context):
        pose = plant.EvalBodyPoseInWorld(context, gripper)   ## This is the important line
        xyz.value = np.array2string(pose.translation(), formatter={'float': lambda x: "{:3.2f}".format(x)})
        rpy.value = np.array2string(RollPitchYaw(pose.rotation()).vector(), formatter={'float': lambda x: "{:3.2f}".format(x)})

    meshcat.load()
    MakeJointSlidersThatPublishOnCallback(station.get_multibody_plant(), meshcat, context, my_callback=pose_callback)
    display(xyz)
    display(rpy)

gripper_forward_kinematics_example()

NameError: name 'zmq_url' is not defined